In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
from sklearn.metrics import roc_auc_score
import cv2
import keras.backend as K
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
import xgboost as xgb
from keras.models import Sequential
%matplotlib inline

Using TensorFlow backend.


In [2]:
X_train=np.load('X_train.npy', mmap_mode='r+')
Y_train=np.load('Y_train.npy', mmap_mode='r+')
X_test=np.load('X_test.npy',mmap_mode='r+')
Y_test=np.load('Y_test.npy',mmap_mode='r+')

In [3]:
def unweighted_binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred))

In [4]:
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')
new_preds = Dense(15,activation='sigmoid')(dense_model.output)
pad_model = Model(dense_model.input,new_preds)
pad_model.compile(optimizer='rmsprop',loss=unweighted_binary_crossentropy,metrics=['accuracy'])
pad_model.load_weights('weights.hdf5')

In [5]:
pad_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [6]:
intermediate_layer_model = Model(inputs=pad_model.input,
                                 outputs=pad_model.get_layer('relu').output)
intermediate_train_output = intermediate_layer_model.predict(X_train)
intermediate_test_output = intermediate_layer_model.predict(X_test)

In [7]:
flattened_model=Sequential()
flattened_model.add(Flatten())
flattened_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
feat_train = flattened_model.predict(intermediate_train_output)
print(feat_train.shape)

(13118, 50176)


In [9]:
feat_test = flattened_model.predict(intermediate_test_output)
print(feat_test.shape)

(12815, 50176)


In [10]:
dtrain = xgb.DMatrix(feat_train, label=Y_train)
dtest = xgb.DMatrix(feat_test, label=Y_test)
param = { 'max_depth': 3,'eta': 0.3, 'silent': 0, 'objective': 'multi:softprob', 'num_class': 15}
num_round = 1  # the number of training iterations

In [11]:
del intermediate_layer_model
del intermediate_train_output
del intermediate_test_output
del flattened_model
del dense_model
del pad_model

In [12]:
np.save('./feat_test.npy', feat_test)
np.save('./feat_train.npy', feat_train)

In [ ]:
dtrain.save_binary('dmatrix_train.txt')
dtest.save_binary('dmatrix_test.txt')

In [ ]:
#xb = xgb.XGBClassifier()
#xb.fit(feat_train,Y_train)
xgboost_model = xgb.train(param, dtrain, num_round)

In [18]:
xb.score(feat_test,np.argmax(Y_test,axis=1))

0.6906749902458057

In [33]:
Pred_labels = xb.predict(dtest)

TypeError: can not initialize DMatrix from DMatrix

In [22]:
Pred_labels[0]

0

In [27]:
dtrain = xgb.DMatrix(X_train, label=Y_train)

ValueError: Input numpy.ndarray must be 2 dimensional